In [1]:
import nltk
import sqlite3
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd





In [2]:
print("Loading Data From DB")

targetDatabase = 'eng1B.db'
con = sqlite3.connect(targetDatabase)
db = pd.read_sql_query("""SELECT sid, cid, clemma, tag, tags FROM concept""",con)

print("Data Loaded!")



Loading Data From DB
Data Loaded!


In [84]:
# DataSanitization

# TODO: Implement this based on the data
db.head(5)


,sid,cid,clemma,tag,tags
0,60000,0,prime,x,01012990-a; 01489557-a; 01013171-a; 02342899-a; 03012209-a; 15152409-n; 15295045-n; 15229784-n; 13594005-n; 01365355-v; 00408624-v; 00454018-v
1,60000,1,28,e,02189108-a; 13749146-n
2,60000,2,hold,01733477-v,13999663-n; 02964634-n; 15272029-n; 03485997-n; 03525372-n; 05806623-n; 00812526-n; 03525252-n; 05197232-n; 02706816-v; 01733477-v; 02681795-v; 02732798-v; 00736586-v; 01216670-v; 02203362-v; 02700867-v; 01859586-v; 00607000-v; 02746275-v; 01334744-v; 00683771-v; 02676789-v; 02683489-v; 00805376-v; 02701210-v; 02283324-v; 02498320-v; 00885217-v; 02450989-v; 01601234-v; 02510337-v; 01773130-v; 01301410-v; 01217043-v; 01151753-v; 02648502-v; 00004492-v; 02643574-v; 00822367-v; 01205153-v; 02441897-v; 02302220-v; 01129876-v; 00693780-v
3,60000,3,press,06263369-n,06263369-n; 03999992-n; 08183398-n; 04550184-n; 00627013-n; 04000311-n; 04000480-n; 00113113-n; 14451349-n; 00759657-v; 01754105-v; 01150370-v; 01447632-v; 01390616-v; 00765649-v; 02589576-v; 01387786-v; 02024367-v; 01662614-v; 01447257-v; 02722782-v; 01390833-v; 02646064-v
4,60000,4,conference,08308497-n,07142566-n; 08308497-n; 08231184-n


In [3]:
# for each word, how many instance
#  for each word instance, list tags
#   for each tags, how many times the tag appear in the word

def tagsCount(row):
    tagsString = row['tags']
    if tagsString == None:
        return 0
    else:
        return len(row['tags'].split(";"))

db['tagsCount'] =  db.apply(tagsCount, axis=1)
# db.head(20)
print("Done!")

In [4]:
numWords = db.groupby('clemma').size()
wordsDB = db.drop(['sid', 'cid','tag'], axis = 1)
# wordsDB.head(5)
numWords = numWords.to_frame()
numWords.reset_index(level=0, inplace=True)
numWords.rename(columns={ numWords.columns[0]: "clemma" , numWords.columns[1]: 'wordCount'}, inplace = True)
numWords.head(5)
print("Done!")

,clemma,wordCount
0,!,1
1,#39,1
2,$,4
3,%,5
4,',18


In [5]:
allTags = db.drop(['clemma','sid', 'cid','tags','tagsCount'], axis = 1)
allTags = allTags.drop_duplicates()

allWords = db.drop(['tag','sid', 'cid','tags','tagsCount'], axis = 1)
allWords = allWords.drop_duplicates()
allWords

allWordCounts = db.groupby(['clemma']).size()
print("Done!")

In [6]:

wordsDB = pd.merge(wordsDB,numWords, on="clemma")
wordsDB = wordsDB.drop_duplicates()
# wordsDB.head(10)
def stringToList(string):
    if string == None:
        return []
    else:
        return string.split(';')

wordsDB['tags'] = wordsDB.apply(lambda x : stringToList(x['tags']), axis =1  )

wordsDB
a= wordsDB.groupby(['clemma']).agg({'tags': "sum"})
a.reset_index(level=0, inplace=True)
a['tags'] = a.apply(lambda r: set(r['tags']), axis = 1)
a['tagsCount'] = a.apply(lambda r: len(r['tags']), axis = 1)
a = a .sort_values(['tagsCount'], ascending = False )
a
print("Done!")

,clemma,tags,tagsCount
3973,break,"{ 07313814-n, 02030764-v, 00778745-v, 00059...",79
6764,cut,"{ 14429885-n, 00489475-n, 00096136-v, 01611...",74
20832,run,"{ 02099829-v, 01914947-v, 00444629-v, 01525...",60
21594,setting,"{ 07999699-n, 06199702-n, 07363545-n, 01494...",55
18587,play,"{ 04774511-n, 01080064-v, 01725051-v, 000136...",55
...,...,...,...
8290,eat into,{},0
8292,eat out,{},0
8302,ebbe,{},0
8315,eco,{},0


In [7]:
tagCount=0

def getTagCount(row):
    global tagCount
    tagCount+=1      
    word = row['clemma']
    print(word, tagCount*100/len(allWords), "percent Complete")
    tags = dict.fromkeys(row['tags'], 0)
    strippedTags = {}
    s = 0
    sumTags=0
    tags['w'] = 0
    tags['x'] = 0
    tags['pers'] = 0
    for tag in tags.keys():
        # TODO: CHANGED HERE
        strippedTag = tag.strip()
        count = len(db[ (db['clemma'] == word) & ( db['tag'] == strippedTag) ].index)
        strippedTags[strippedTag] = count
        sumTags+=count
        n = count/allWordCounts[word]
        if n == 0:
            continue
        nlogn = n * math.log10(n)
        # tags[tag] = nlogn
        s += abs(nlogn)

    if (s>1):
        print("ANOMALY!", word )
        print("word Count:",allWordCounts[word], "tag: ",sumTags )
    return (s,strippedTags)

b  = a.apply(getTagCount, axis =1)
# len(db[ (db['clemma'] == word) & ( db['tags'].str.contains(tag)) ])
# a['tagWithCounts'] = a.apply(getTagCount, axis =1)
#d=  a.head(10)
        

nt Complete
securities market 98.4471257244924 percent Complete
sections 98.4507709692706 percent Complete
secretary of state 98.45441621404878 percent Complete
secret society 98.45806145882696 percent Complete
secret code 98.46170670360515 percent Complete
secret agent 98.46535194838333 percent Complete
dispense with 98.46899719316153 percent Complete
disposable income 98.47264243793971 percent Complete
second thought 98.4762876827179 percent Complete
second person 98.47993292749608 percent Complete
second nature 98.48357817227426 percent Complete
second half 98.48722341705246 percent Complete
dispose of 98.49086866183065 percent Complete
seater 98.49451390660883 percent Complete
seared 98.49815915138701 percent Complete
dissolute 98.5018043961652 percent Complete
seal in 98.50544964094338 percent Complete
divided highway 98.50909488572158 percent Complete
dizzying 98.51274013049976 percent Complete
djb 98.51638537527795 percent Complete
djs 98.52003062005613 percent Complete
sea capt

In [21]:
a["results"] = b
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a

,clemma,tags,tagsCount,results
3973,break,"{ 07313814-n, 02030764-v, 00778745-v, 00059989-n, 01610666-v, 02535716-v, 00376400-n, 00935987-v, 00364064-v, 00558536-v, 00779061-v, 00434374-v, 01610463-v, 15271008-n, 02681639-v, 00383952-n, 00188007-n, 07367812-n, 01228524-v, 02029369-v, 14292090-n, 01557646-v, 01991347-v, 00336260-v, 02101373-v, 01106864-v, 02746617-v, 02431320-v, 02318165-v, 00258665-v, 80001019-v, 01228396-v, 02668523-v, 00376400-n, 01369758-v, 00390215-v, 02073233-v, 02399331-v, 02570684-v, 00738440-v, 01972976-v, 00030463-v, 02600082-v, 01989053-v, 02681524-v, 00500280-n, 02746449-v, 02625187-v, 09278537-n, 02478469-v, 01785395-v, 00258366-v, 00202569-v, 00738314-v, 07367548-n, 07297633-n, 00294748-n, 00294748-n, 07316856-n, 02566528-v, 02559395-v, 00107739-v, 01597995-v, 00334996-v, 00551412-v, 00482892-n, 01896484-v, 01062997-n, 00339464-v, 00155046-v, 01298931-v, 00933821-v, 00202784-v, 00744439-v, 02570684-v, 00107604-v, 00362348-v, 00334186-v, 01369346-v}",79,"(0.2618605995406722, {'07313814-n': 1, '02030764-v': 0, '00778745-v': 0, '00059989-n': 0, '01610666-v': 0, '02535716-v': 3, '00376400-n': 0, '00935987-v': 0, '00364064-v': 2, '00558536-v': 1, '00779061-v': 0, '00434374-v': 0, '01610463-v': 0, '15271008-n': 1, '02681639-v': 0, '00383952-n': 0, '00188007-n': 0, '07367812-n': 1, '01228524-v': 0, '02029369-v': 0, '14292090-n': 0, '01557646-v': 0, '01991347-v': 0, '00336260-v': 0, '02101373-v': 0, '01106864-v': 3, '02746617-v': 0, '02431320-v': 1, '02318165-v': 0, '00258665-v': 0, '80001019-v': 0, '01228396-v': 0, '02668523-v': 0, '01369758-v': 0, '00390215-v': 0, '02073233-v': 0, '02399331-v': 0, '02570684-v': 1, '00738440-v': 0, '01972976-v': 0, '00030463-v': 0, '02600082-v': 1, '01989053-v': 1, '02681524-v': 0, '00500280-n': 0, '02746449-v': 0, '02625187-v': 0, '09278537-n': 0, '02478469-v': 0, '01785395-v': 0, '00258366-v': 0, '00202569-v': 0, '00738314-v': 0, '07367548-n': 0, '07297633-n': 0, '00294748-n': 0, '07316856-n': 0, '02566528-v': 1, '02559395-v': 1, '00107739-v': 0, '01597995-v': 0, '00334996-v': 1, '00551412-v': 0, '00482892-n': 0, '01896484-v': 0, '01062997-n': 3, '00339464-v': 0, '00155046-v': 0, '01298931-v': 0, '00933821-v': 1, '00202784-v': 0, '00744439-v': 0, '00107604-v': 0, '00362348-v': 0, '00334186-v': 2, '01369346-v': 0, 'w': 0, 'x': 14, 'pers': 0})"
6764,cut,"{ 14429885-n, 00489475-n, 00096136-v, 01611123-v, 06612865-n, 01754737-v, 01319738-v, 00352331-n, 00243900-v, 01320424-v, 00429968-v, 06721461-n, 00429060-v, 00108475-v, 14287113-n, 02101649-v, 00004615-a, 03152619-n, 01756563-v, 02422283-v, 06428216-n, 01754915-v, 01756277-v, 00882742-a, 07653394-n, 01610990-v, 02454750-a, 00201722-v, 02137070-a, 01555437-v, 02588280-v, 01225997-n, 00661278-a, 01610834-v, 01756416-v, 01059564-v, 01860107-v, 01510576-v, 01552519-v, 00756327-a, 14287113-n, 00662958-a, 05750948-n, 00359238-n, 00447654-v, 00571444-n, 00386915-n, 01413917-v, 01713491-v, 01666717-v, 02033295-v, 06618234-n, 02134232-v, 13289159-n, 00387657-n, 02422283-v, 00473322-v, 01915730-v, 01859995-v, 00292507-v, 00067254-n, 00662958-a, 02454750-a, 00565809-n, 00226071-v, 01566027-a, 01064799-v, 02613860-v, 00663267-a, 01915865-v, 00561090-v, 01227088-v, 03152743-n, 00061219-v}",74,"(0.25637868926095153, {'14429885-n': 0, '00489475-n': 0, '00096136-v': 0, '01611123-v': 0, '06612865-n': 0, '01754737-v': 1, '01319738-v': 0, '00352331-n': 0, '00243900-v': 0, '01320424-v': 1, '00429968-v': 0, '06721461-n': 0, '00429060-v': 8, '00108475-v': 0, '14287113-n': 0, '02101649-v': 0, '00004615-a': 0, '03152619-n': 0, '01756563-v': 0, '02422283-v': 0, '06428216-n': 0, '01754915-v': 0, '01756277-v': 0, '00882742-a': 0, '07653394-n': 0, '01610990-v': 0, '02454750-a': 0, '00201722-v': 0, '02137070-a': 0, '01555437-v': 0, '02588280-v': 0, '01225997-n': 0, '00661278-a': 1, '01610834-v': 0, '01756416-v': 0, '01059564-v': 0, '01860107-v': 0, '01510576-v': 0, '01552519-v': 1, '00756327-a': 0, '00662958-a': 0, '05750948-n': 1, '00359238-n': 0,

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a[ (a['clemma'] == "break" )]

In [22]:
import os
print(os.getcwd())
p = os.path.join(os.getcwd(), "PicklepeePumperrump.pckl")
print(p)
a.to_pickle(p)

c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs
c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs\PicklepeePumperrump.pckl


In [4]:
print("Unpickling!")

rdb = pd.read_pickle('2016hg8011_eng1B_processed.pckl')
print("Unpickled!")
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 0)
rdb

Unpickling!
Unpickled!


,clemma,tags,tagsCount,results
3973,break,"{ 01062997-n, 00258366-v, 02431320-v, 01972976-v, 00202784-v, 02030764-v, 02073233-v, 01369758-v, 02681639-v, 07313814-n, 02625187-v, 00364064-v, 00030463-v, 01557646-v, 00334186-v, 00294748-n, 14292090-n, 07316856-n, 02478469-v, 02535716-v, 15271008-n, 00334996-v, 01991347-v, 00558536-v, 07367812-n, 00935987-v, 01610463-v, 01228396-v, 07297633-n, 00434374-v, 01989053-v, 00779061-v, 00390215-v, 00744439-v, 00738314-v, 00258665-v, 00376400-n, 80001019-v, 01785395-v, 00107604-v, 02746449-v, 01369346-v, 01610666-v, 00362348-v, 02101373-v, 02399331-v, 00933821-v, 00482892-n, 09278537-n, 01896484-v, 00059989-n, 00738440-v, 02681524-v, 02570684-v, 02746617-v, 00188007-n, 00336260-v, 00202569-v, 02668523-v, 00376400-n, 01298931-v, 00383952-n, 00155046-v, 02318165-v, 02566528-v, 01597995-v, 07367548-n, 01228524-v, 02600082-v, 00339464-v, 01106864-v, 02570684-v, 00294748-n, 00551412-v, 00500280-n, 02559395-v, 00107739-v, 02029369-v, 00778745-v}",79,"(0.2618605995406722, {'07313814-n': 1, '02030764-v': 0, '00778745-v': 0, '00059989-n': 0, '01610666-v': 0, '02535716-v': 3, '00376400-n': 0, '00935987-v': 0, '00364064-v': 2, '00558536-v': 1, '00779061-v': 0, '00434374-v': 0, '01610463-v': 0, '15271008-n': 1, '02681639-v': 0, '00383952-n': 0, '00188007-n': 0, '07367812-n': 1, '01228524-v': 0, '02029369-v': 0, '14292090-n': 0, '01557646-v': 0, '01991347-v': 0, '00336260-v': 0, '02101373-v': 0, '01106864-v': 3, '02746617-v': 0, '02431320-v': 1, '02318165-v': 0, '00258665-v': 0, '80001019-v': 0, '01228396-v': 0, '02668523-v': 0, '01369758-v': 0, '00390215-v': 0, '02073233-v': 0, '02399331-v': 0, '02570684-v': 1, '00738440-v': 0, '01972976-v': 0, '00030463-v': 0, '02600082-v': 1, '01989053-v': 1, '02681524-v': 0, '00500280-n': 0, '02746449-v': 0, '02625187-v': 0, '09278537-n': 0, '02478469-v': 0, '01785395-v': 0, '00258366-v': 0, '00202569-v': 0, '00738314-v': 0, '07367548-n': 0, '07297633-n': 0, '00294748-n': 0, '07316856-n': 0, '02566528-v': 1, '02559395-v': 1, '00107739-v': 0, '01597995-v': 0, '00334996-v': 1, '00551412-v': 0, '00482892-n': 0, '01896484-v': 0, '01062997-n': 3, '00339464-v': 0, '00155046-v': 0, '01298931-v': 0, '00933821-v': 1, '00202784-v': 0, '00744439-v': 0, '00107604-v': 0, '00362348-v': 0, '00334186-v': 2, '01369346-v': 0, 'w': 0, 'x': 14, 'pers': 0})"
6764,cut,"{ 01754915-v, 01756416-v, 00571444-n, 00561090-v, 06428216-n, 01413917-v, 01064799-v, 00352331-n, 07653394-n, 03152619-n, 00201722-v, 00386915-n, 01319738-v, 02588280-v, 01227088-v, 02101649-v, 01666717-v, 01225997-n, 01860107-v, 01510576-v, 00663267-a, 00662958-a, 02422283-v, 01611123-v, 01566027-a, 00447654-v, 00096136-v, 03152743-n, 01610834-v, 00661278-a, 01915730-v, 00429968-v, 14287113-n, 00756327-a, 02422283-v, 01754737-v, 01859995-v, 06721461-n, 00882742-a, 01756563-v, 01555437-v, 00243900-v, 00226071-v, 14429885-n, 02454750-a, 01320424-v, 00489475-n, 01915865-v, 01552519-v, 02137070-a, 00565809-n, 00429060-v, 13289159-n, 00387657-n, 00359238-n, 00067254-n, 00061219-v, 02033295-v, 06618234-n, 06612865-n, 01610990-v, 01756277-v, 02613860-v, 02134232-v, 14287113-n, 01059564-v, 05750948-n, 00292507-v, 00004615-a, 00473322-v, 00662958-a, 00108475-v, 02454750-a, 01713491-v}",74,"(0.25637868926095153, {'14429885-n': 0, '00489475-n': 0, '00096136-v': 0, '01611123-v': 0, '06612865-n': 0, '01754737-v': 1, '01319738-v': 0, '00352331-n': 0, '00243900-v': 0, '01320424-v': 1, '00429968-v': 0, '06721461-n': 0, '00429060-v': 8, '00108475-v': 0, '14287113-n': 0, '02101649-v': 0, '00004615-a': 0, '03152619-n': 0, '01756563-v': 0, '02422283-v': 0, '06428216-n': 0, '01754915-v': 0, '01756277-v': 0, '00882742-a': 0, '07653394-n': 0, '01610990-v': 0, '02454750-a': 0, '00201722-v': 0, '02137070-a': 0, '01555437-v': 0, '02588280-v': 0, '01225997-n': 0, '00661278-a': 1, '01610834-v': 0, '01756416-v': 0, '01059564-v': 0, '01860107-v': 0, '01510576-v': 0, '01552519-v': 1, '00756327-a': 0, '00662958-a': 0, '05750948-n': 1, '00359238-n': 0,

In [41]:
rdb['entropyVal'] =  rdb.apply(lambda x: x['results'][0], axis =1)
rdb = rdb.sort_values(['entropyVal'],axis = 0, ascending= False )
rdb

,clemma,tags,tagsCount,results,entropyVal
24912,total,"{ 00949288-v, 04353803-n, 02645007-v, 00522885-a, 05861067-n, 00515380-a, 00522885-a, 00515380-a, 04353803-n, 00259382-v}",10,"(1.0178058215066794, {'00259382-v': 0, '04353803-n': 9, '00522885-a': 4, '00515380-a': 10, '02645007-v': 3, '05861067-n': 10, '00949288-v': 0, 'w': 0, 'x': 0, 'pers': 0})",1.017806
24927,tough,"{ 09879297-n, 00478311-a, 02512641-a, 00746451-a, 02445636-a, 00707795-a, 01127302-a, 00746451-a, 02448437-a, 10662649-n, 02447344-a, 10184081-n, 02448437-a, 00748058-a}",14,"(0.9264783457485914, {'00748058-a': 1, '09879297-n': 0, '02448437-a': 3, '00707795-a': 0, '00746451-a': 2, '02512641-a': 0, '00478311-a': 0, '02445636-a': 0, '10184081-n': 0, '10662649-n': 0, '01127302-a': 1, '02447344-a': 0, 'w': 0, 'x': 1, 'pers': 0})",0.926478
19241,programme,"{ 06748466-n, 00551215-n, 06619428-n, 01747717-v, 06748466-n, 06568978-n, 05899087-n, 06676416-n, 00795264-v, 05899087-n, 05898568-n}",11,"(0.9261725217154378, {'06676416-n': 2, '00551215-n': 2, '06748466-n': 1, '05898568-n': 1, '00795264-v': 2, '05899087-n': 3, '01747717-v': 0, '06568978-n': 0, '06619428-n': 0, 'w': 0, 'x': 0, 'pers': 0})",0.926173
20453,review,"{00855512-v, 13411533-n, 06410391-n, 00696852-v, 00696700-v, 06410391-n, 00897811-n, 00855512-v, 07019866-n, 06469377-n, 01197258-n, 00696414-v, 05747582-n, 07019866-n, 06597758-n, 00143251-n, 00696189-v, 00879271-n}",18,"(0.9071580364883152, {'05747582-n': 5, '07019866-n': 0, '06597758-n': 0, '00855512-v': 3, '00696852-v': 0, '00897811-n': 0, '00879271-n': 2, '06469377-n': 0, '00143251-n': 0, '00696189-v': 5, '13411533-n': 0, '06410391-n': 2, '00696700-v': 0, '00696414-v': 2, '01197258-n': 0, 'w': 0, 'x': 0, 'pers': 0})",0.907158
16620,national,"{ 02988060-a, 01038332-a, 09625401-n, 09625401-n, 01105840-a, 01567862-a, 01861776-a, 02988281-a, 02988402-a, 01861776-a, 01105840-a}",11,"(0.8979169640013727, {'01105840-a': 14, '02988281-a': 0, '09625401-n': 2, '01038332-a': 0, '02988402-a': 0, '01861776-a': 4, '01567862-a': 4, '02988060-a': 10, 'w': 0, 'x': 1, 'pers': 0})",0.897917
7298,design,"{ 00698572-v, 05633385-n, 00709625-v, 00928077-n, 05728678-n, 01638368-v, 01638368-v, 03178782-n, 01633825-v, 05982152-n, 01640550-v, 05633385-n, 01639714-v, 03179318-n, 05902327-n, 01753596-v}",16,"(0.8961212871443924, {'00928077-n': 8, '00698572-v': 4, '03179318-n': 2, '01639714-v': 8, '03178782-n': 29, '00709625-v': 3, '01633825-v': 0, '05902327-n': 1, '05633385-n': 6, '01753596-v': 0, '01640550-v': 11, '01638368-v': 3, '05728678-n': 3, '05982152-n': 1, 'w': 0, 'x': 0, 'pers': 0})",0.896121
9021,expand,"{00257269-v, 00541163-v, 02077148-v, 00257269-v, 00540946-v, 00310386-v, 00955601-v, 02077148-v, 00264386-v}",9,"(0.8945009309194077, {'02077148-v': 1, '00540946-v': 2, '00264386-v': 1, '00257269-v': 3, '00310386-v': 2, '00955601-v': 0, '00541163-v': 2, 'w': 0, 'x': 0, 'pers': 0})",0.894501
6481,create,"{ 01753788-v, 02476385-v, 01640207-v, 01685313-v, 01617192-v, 01621555-v, 01617192-v, 01640207-v}",8,"(0.8540481078096791, {'01617192-v': 19, '01640207-v': 11, '01621555-v': 4, '01753788-v': 21, '02476385-v': 0, '01685313-v': 1, 'w': 0, 'x': 0, 'pers': 0})",0.854048
9039,experience,"{ 00596644-v, 00121046-v, 01771535-v, 05984584-n, 05758059-n, 01771535-v, 07285403-n, 02108026-v, 02110220-v, 07285403-n, 05984584-n}",11,"(0.8411185311712834, {'05758059-n': 12, '05984584-n': 49, '07285403-n': 36, '02108026-v': 28, '02110220-v': 6, '01771535-v': 1, '00121046-v': 1, '00596644-v': 14, 'w': 0, 'x': 0, 'pers': 0})",0.841119
...,...,...,...,...,...
